# Project1 (Veronica): regime-dependent uncertainty σ(s)
This notebook runs E1/E2/E3 on top of the replication code.

Assumptions:
- You already have baseline trained runs saved in `../artifacts/selected_runs.json` (from replication or fast retrain notebook).
- We **do not** retrain for E2; E3 is optional retraining.


In [ ]:

import os, sys, json, numpy as np, pandas as pd, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer, simulate_paths
from src.io_utils import load_json, ensure_dir, save_csv
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, solve_taylor_sss, solve_discretion_sss, solve_commitment_sss
from src.flex_stochastic import simulate_flex_and_rstar


In [ ]:

ARTIFACTS = "../artifacts"
ensure_dir(os.path.join(ARTIFACTS, "results"))
device="cpu"
dtype=torch.float32
params0 = ModelParams(device=device, dtype=dtype)  # baseline


## Load baseline selected runs

In [ ]:

selected = load_json(os.path.join(ARTIFACTS, "selected_runs.json"))
selected


In [ ]:

DIMS = {"taylor":(5,8), "mod_taylor":(5,8), "discretion":(5,11), "commitment":(7,13)}

def load_net(policy: str):
    run_dir = selected[policy]["run_dir"]
    cfg = json.load(open(os.path.join(run_dir, "config.json"), "r"))
    d_in, d_out = DIMS[policy]
    net = PolicyNetwork(d_in=d_in, d_out=d_out,
                        hidden=tuple(cfg.get("hidden_layers",(512,512))),
                        activation=cfg.get("activation","selu"))
    wname = selected[policy].get("best_weights_name","weights_best.pt")
    wpath = os.path.join(run_dir, wname)
    net.load_state_dict(torch.load(wpath, map_location="cpu"))
    net.eval()
    return net, run_dir

net_taylor, rd_taylor = load_net("taylor")
net_mod, rd_mod = load_net("mod_taylor")
print("Taylor:", rd_taylor)
print("ModTaylor:", rd_mod)


## Build baseline r̄ by regime (for modified Taylor)

In [ ]:

flex = solve_flexprice_sss(params0)
rbar = export_rbar_tensor(flex, params0)
rbar


## E1 — stochastic r* sweep (flex-price)

In [ ]:

k_grid = [1.0, 1.25, 1.5, 2.0, 3.0]
rows=[]
for k in k_grid:
    est = simulate_flex_and_rstar(params0, T=120_000, burn_in=20_000, seed=123, sigma_bad_mult=k)
    rows.append({"k":k, **est})
    print("k=",k, est)

df_r = pd.DataFrame(rows)
df_r.to_csv(os.path.join(ARTIFACTS,"results","project1_E1_rstar_sweep.csv"), index=False)
df_r


In [ ]:

plt.figure()
plt.plot(df_r["k"], 100*df_r["delta_rstar"])
plt.xlabel("sigma_bad_mult k")
plt.ylabel("100 × (r*_bad − r*_normal)")
plt.title("E1: Veronica channel — Δr*(k)")
plt.show()


## E2 — policy recipe test (no retraining)
We change only σ(s) (bad-regime volatility) and resimulate using baseline nets.
We report regime-conditional mean/std for π, i, y, c.

In [ ]:

def make_params_k(k: float):
    return ModelParams(**{**params0.__dict__,
                         "sigma_A_bad_mult":k,
                         "sigma_g_bad_mult":k,
                         "sigma_tau_bad_mult":k}).to_torch()

def regime_stats(store):
    s = store["s"].reshape(-1)
    out={}
    for key in ["pi","i","y","c"]:
        x = store[key].reshape(-1)
        out[f"{key}_mean_s0"]=float(np.mean(x[s==0]))
        out[f"{key}_mean_s1"]=float(np.mean(x[s==1]))
        out[f"{key}_std_s0"]=float(np.std(x[s==0]))
        out[f"{key}_std_s1"]=float(np.std(x[s==1]))
    return out

# fixed x0 for comparability
B = 256
tmp_tr = Trainer(params=params0, cfg=TrainConfig(seed=123), policy="taylor", net=net_taylor, gh_n=3)
x0 = tmp_tr.simulate_initial_state(B)

rows=[]
for k in k_grid:
    pk = make_params_k(k)

    st_t = simulate_paths(pk, "taylor", net_taylor, T=40_000, burn_in=5_000, x0=x0, gh_n=7)
    st_m = simulate_paths(pk, "mod_taylor", net_mod, T=40_000, burn_in=5_000, x0=x0, rbar_by_regime=rbar, gh_n=7)

    r = {"k":k}
    r.update({f"taylor_{kk}":vv for kk,vv in regime_stats(st_t).items()})
    r.update({f"modtaylor_{kk}":vv for kk,vv in regime_stats(st_m).items()})
    rows.append(r)
    print("done k",k)

df_e2 = pd.DataFrame(rows)
df_e2.to_csv(os.path.join(ARTIFACTS,"results","project1_E2_policy_recipe.csv"), index=False)
df_e2.head()


In [ ]:

# Plot regime-mean inflation vs k
plt.figure()
plt.plot(df_e2["k"], df_e2["taylor_pi_mean_s0"], label="Taylor: mean π | s=0")
plt.plot(df_e2["k"], df_e2["taylor_pi_mean_s1"], label="Taylor: mean π | s=1")
plt.plot(df_e2["k"], df_e2["modtaylor_pi_mean_s0"], label="ModTaylor: mean π | s=0")
plt.plot(df_e2["k"], df_e2["modtaylor_pi_mean_s1"], label="ModTaylor: mean π | s=1")
plt.xlabel("sigma_bad_mult k")
plt.ylabel("mean inflation π")
plt.title("E2: policy recipe (no retraining) — regime means vs k")
plt.legend()
plt.show()


## E2 visual — Common random numbers overlay (k=1 vs k=2)
We fix the same initial state, the same Markov draws and the same standard-normal shocks.
Only σ(s) changes. This gives a clean 'apples-to-apples' visual comparison.

In [ ]:

from src.project1_experiments import simulate_paths_crn

T=600
burn_in=0
B=64
seed=777

# fixed initial state batch
tmp_tr = Trainer(params=params0, cfg=TrainConfig(seed=seed), policy="taylor", net=net_taylor, gh_n=3)
x0_crn = tmp_tr.simulate_initial_state(B)

# generate common shocks
rng = np.random.default_rng(seed)
epsA = rng.standard_normal((T,B)).astype(np.float32)
epsg = rng.standard_normal((T,B)).astype(np.float32)
epst = rng.standard_normal((T,B)).astype(np.float32)
u = rng.random((T,B)).astype(np.float32)  # for Markov transitions

shocks = {"epsA":epsA, "epsg":epsg, "epst":epst, "u":u}

def run_overlay(k):
    pk = make_params_k(k)
    out_t = simulate_paths_crn(pk, "taylor", net_taylor, T=T, burn_in=burn_in, x0=x0_crn, shocks=shocks, rbar_by_regime=None)
    out_m = simulate_paths_crn(pk, "mod_taylor", net_mod, T=T, burn_in=burn_in, x0=x0_crn, shocks=shocks, rbar_by_regime=rbar)
    return out_t, out_m

t1, m1 = run_overlay(1.0)
t2, m2 = run_overlay(2.0)

def plot_overlay(var, title):
    plt.figure()
    plt.plot(t1[var][:,0], label=f"Taylor k=1")
    plt.plot(t2[var][:,0], label=f"Taylor k=2")
    plt.plot(m1[var][:,0], label=f"ModTaylor k=1")
    plt.plot(m2[var][:,0], label=f"ModTaylor k=2")
    plt.title(title)
    plt.xlabel("t")
    plt.legend()
    plt.show()

plot_overlay("pi", "Inflation π — CRN overlay (same shocks)")
plot_overlay("i", "Nominal rate i — CRN overlay (same shocks)")
plot_overlay("y", "Output y — CRN overlay (same shocks)")
plot_overlay("c", "Consumption c — CRN overlay (same shocks)")


## E2 additional — Regime distributions (histograms) and tail quantiles (k=1 vs k=2)
This compares the *distribution* of key variables within each regime under baseline (k=1) and higher bad-regime uncertainty (k=2).
We report mean/std and selected quantiles (5%, 50%, 95%) and plot histograms for inflation.


In [ ]:
k1, k2 = 1.0, 2.0
T_dist = 60_000
burn_dist = 5_000
B_dist = 256

# Reuse the same initial state for fair comparison
tmp_tr = Trainer(params=params0, cfg=TrainConfig(seed=321), policy="taylor", net=net_taylor, gh_n=3)
x0_dist = tmp_tr.simulate_initial_state(B_dist)

def sim_store(policy, k):
    pk = make_params_k(k)
    if policy == "taylor":
        return simulate_paths(pk, "taylor", net_taylor, T=T_dist, burn_in=burn_dist, x0=x0_dist, gh_n=7)
    if policy == "mod_taylor":
        return simulate_paths(pk, "mod_taylor", net_mod, T=T_dist, burn_in=burn_dist, x0=x0_dist, rbar_by_regime=rbar, gh_n=7)
    raise ValueError(policy)

stores = {
    ("taylor", k1): sim_store("taylor", k1),
    ("taylor", k2): sim_store("taylor", k2),
    ("mod_taylor", k1): sim_store("mod_taylor", k1),
    ("mod_taylor", k2): sim_store("mod_taylor", k2),
}

def summarize_quantiles(store, *, policy, k):
    s = store["s"].reshape(-1)
    rows=[]
    for var in ["pi","i","y","c"]:
        x = store[var].reshape(-1)
        for reg in [0,1]:
            xr = x[s==reg]
            rows.append({
                "policy": policy,
                "k": float(k),
                "regime": int(reg),
                "var": var,
                "mean": float(np.mean(xr)),
                "std": float(np.std(xr)),
                "q05": float(np.quantile(xr, 0.05)),
                "q50": float(np.quantile(xr, 0.50)),
                "q95": float(np.quantile(xr, 0.95)),
            })
    return rows

rows=[]
for (pol,k), st in stores.items():
    rows += summarize_quantiles(st, policy=pol, k=k)

df_q = pd.DataFrame(rows)
out_csv = os.path.join(ARTIFACTS, "results", "project1_E2_quantiles_k1_vs_k2.csv")
df_q.to_csv(out_csv, index=False)
print("Saved:", out_csv)
df_q.sort_values(["var","policy","regime","k"]).head(12)

# Histograms: inflation distribution by regime (overlay k=1 vs k=2) for each policy
def plot_hist_pi(policy):
    for reg in [0,1]:
        plt.figure()
        for k in [k1, k2]:
            st = stores[(policy, k)]
            s = st["s"].reshape(-1)
            x = st["pi"].reshape(-1)
            xr = x[s==reg]
            plt.hist(xr, bins=60, density=True, alpha=0.5, label=f"k={k}")
        plt.title(f"Inflation π distribution | policy={policy} | regime s={reg} (k=1 vs k=2)")
        plt.xlabel("π")
        plt.ylabel("density")
        plt.legend()
        plt.show()

plot_hist_pi("taylor")
plot_hist_pi("mod_taylor")


## E3 — Retrain across k (optional)
This section re-trains Taylor and ModTaylor for each k.
It is expensive; start with a smoke test (few steps), then increase steps.
The code below writes `project1_E3_runs.json` and a summary CSV.

In [ ]:

# Toggle:
RUN_E3 = False  # set True to run


In [ ]:

from src.io_utils import make_run_dir, save_run_metadata, save_selected_run, save_json

def train_one_k(policy, k, *, seed=123, tag="project1_E3"):
    pk = make_params_k(k)
    run_dir = make_run_dir(ARTIFACTS, policy=policy, tag=f"{tag}_k{k}", seed=seed)
    cfg = TrainConfig(seed=seed, steps=6000, batch_size=256, lr=1e-5, gh_n_train=3,
                      replay_buffer_size=120_000, replay_refresh_every=80, replay_refresh_steps=60, replay_init_fill_steps=400,
                      val_size=4096, val_every=500, early_stopping=False, artifacts_root=ARTIFACTS, run_dir=run_dir, save_best=True)

    flex = solve_flexprice_sss(pk)
    rbar_k = export_rbar_tensor(flex, pk)

    sss = solve_taylor_sss(pk)
    net = PolicyNetwork(d_in=5, d_out=8, hidden=cfg.hidden_layers, activation=cfg.activation)
    tr = Trainer(params=pk, cfg=cfg, policy=policy, net=net, rbar_by_regime=(rbar_k if policy=="mod_taylor" else None), gh_n=cfg.gh_n_train)
    save_run_metadata(run_dir, {"policy":policy,"k":k, **cfg.__dict__, "model_params": pk.__dict__})
    tr.train(sss=sss)
    return run_dir

if RUN_E3:
    e3_runs = []
    for k in k_grid:
        rd_t = train_one_k("taylor", k)
        rd_m = train_one_k("mod_taylor", k)
        e3_runs.append({"k":k, "taylor_run":rd_t, "modtaylor_run":rd_m})
    save_json(os.path.join(ARTIFACTS,"results","project1_E3_runs.json"), e3_runs)
    e3_runs
